In [23]:
import pandas as pd
import math

In [2]:
temp_Quito = pd.read_csv('TemperaturasRegistradas/temperature_Quito_celsius.csv')
temp_Oslo = pd.read_csv('TemperaturasRegistradas/temperature_Oslo_celsius.csv')
temp_Melbourne = pd.read_csv('TemperaturasRegistradas/temperature_Melbourne_celsius.csv')

In [3]:
promQuito = temp_Quito.sum()/temp_Quito.count()
promOslo = temp_Oslo.sum()/temp_Oslo.count()
promMelbourne = temp_Melbourne.sum()/temp_Melbourne.count()

In [ ]:
i=0
for v in temp_Quito.values.flatten():
    i += (v-promQuito)**2
varianzaQuito = i / (temp_Quito.count()-1)
DesvQuito = math.sqrt(float(varianzaQuito.iloc[0]))
print(DesvQuito)

j=0
for v in temp_Oslo.values.flatten():
    j += (v-promOslo)**2
varianzaOslo = j / (temp_Oslo.count()-1)
DesvOslo = math.sqrt(float(varianzaOslo.iloc[0]))
print(DesvOslo)

k=0
for v in temp_Melbourne.values.flatten():
    k += (v-promMelbourne)**2
varianzaMelbourne = k / (temp_Melbourne.count()-1)
DesvMelbourne = math.sqrt(float(varianzaMelbourne.iloc[0]))
print(DesvMelbourne)



AvgTemperature    1.30169
dtype: float64
1.301690430933042
9.46169397382391
6.4776540492718455
